<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/auto-keras/auto_keras_tabledata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/機械学習練習/MLモデル構築自動化/auto-keras
%ls

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/機械学習練習/MLモデル構築自動化/auto-keras
auto_keras_tabledata.ipynb  structured_data_classifier/  titanic.ipynb


# テーブルデータでauto-kerasを試す

In [34]:
%pip install keras-tuner
%pip install autokeras

In [36]:
%pip show autokeras
%pip show keras-tuner
%pip show tensorflow

Name: autokeras
Version: 1.0.6.dev0
Summary: AutoML for deep learning
Home-page: http://autokeras.com
Author: Data Analytics at Texas A&M (DATA) Lab, Keras Team
Author-email: jhfjhfj1@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, scikit-learn, tensorflow, packaging, pandas
Required-by: 
Name: keras-tuner
Version: 1.0.2rc1.dev0
Summary: Hypertuner for Keras
Home-page: https://github.com/keras-team/keras-tuner
Author: The Keras Tuner authors
Author-email: kerastuner@google.com
License: Apache License 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, terminaltables, tqdm, scipy, future, scikit-learn, requests, colorama, packaging, tabulate
Required-by: 
Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
R

In [47]:
import os

import autokeras as ak
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

## データ読み込み

In [52]:
# データセット
DATA_DIR = '../big_dataset'  #@param {type: "string"}
TRAIN_CSV = 'preprocessed_train.csv'  #@param {type: "string"}
TEST_CSV = 'preprocessed_test.csv'  #@param {type: "string"}

# 予測対象の列名
TARGET_COL = 'class' #@param {type: "string"}
NUM_CLASSES = 2 #@param {type: "number"}

# 探索するモデルの上限数
MAX_TRIALS = 11  #@param {type: "slider", min: 0, max: 100}

In [83]:
train_df = pd.read_csv(os.path.join(DATA_DIR, TRAIN_CSV))

# 特徴量の種類を設定しておく
# (設定しなかった場合stringのみを受け付けるモデルとなってしまったため)
def setup_col_type():
  column_types = {}
  for col in train_df.drop(columns=TARGET_COL).columns:
    column_types[col] = 'numerical'
  return column_types 

column_types = setup_col_type()

### モデルのクラス初期化

In [88]:
# 構造化データを元に分類するモデルクラスの初期化
clf = ak.StructuredDataClassifier(max_trials=MAX_TRIALS,# 探索するモデルの上限数
                                  #column_types=column_types,
                                  seed=7,
                                  overwrite=True) 

### モデルの訓練（探索）

In [89]:
clf.fit(#x=os.path.join(DATA_DIR, TRAIN_CSV),
        x=train_df.drop(columns=TARGET_COL).values,
        y=train_df[TARGET_COL].values,
        epochs=30, # 各モデルで回すエポック数(default:1000, earlystoppingあり)
        callbacks=[EarlyStopping(monitor='val_loss',patience=2)], # kerasモデル同様にcallback処理を設定可能
        validation_split=0.2, # 訓練データの末尾から指定した割合が検証データとなる(default:0.2)
        verbose=1, # 通常のkerasモデルで使われるようなキーワード引数も渡せる
        )

Trial 11 Complete [00h 00m 04s]
val_accuracy: 0.918749988079071

Best val_accuracy So Far: 0.9243749976158142
Total elapsed time: 00h 01m 11s
INFO:tensorflow:Oracle triggered exit
Epoch 1/30
250/250 [==============================] - 0s 1ms/step - loss: 0.4468 - accuracy: 0.8389
Epoch 2/30
250/250 [==============================] - 0s 1ms/step - loss: 0.2238 - accuracy: 0.9235
Epoch 3/30
250/250 [==============================] - 0s 1ms/step - loss: 0.2126 - accuracy: 0.9296
Epoch 4/30
250/250 [==============================] - 0s 1ms/step - loss: 0.2056 - accuracy: 0.9320
Epoch 5/30
250/250 [==============================] - 0s 1ms/step - loss: 0.2003 - accuracy: 0.9335
Epoch 6/30
250/250 [==============================] - 0s 2ms/step - loss: 0.1957 - accuracy: 0.9358
Epoch 7/30
250/250 [==============================] - 0s 1ms/step - loss: 0.1917 - accuracy: 0.9379
Epoch 8/30
250/250 [==============================] - 0s 1ms/step - loss: 0.1885 - accuracy: 0.9386
Epoch 9/30
250/250 [

### テストデータに対する性能

In [90]:
result = clf.evaluate(x=os.path.join(DATA_DIR, TEST_CSV), 
                      y=TARGET_COL)
print('loss:{}, accuracy:{}'.format(result[0], result[1]))

63/63 [==============================] - 0s 1ms/step - loss: 0.2181 - accuracy: 0.9230
loss:0.21807485818862915, accuracy:0.9229999780654907


## ベストなモデルをエクスポート

In [91]:
model = clf.export_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 13)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                448       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                

In [92]:
test_df = pd.read_csv(os.path.join(DATA_DIR, TEST_CSV))

test_X = test_df.drop(columns=TARGET_COL).values
test_y = tf.keras.utils.to_categorical(test_df[TARGET_COL].values)